In [1]:
import load_corpus
import generate_CBOW_ds
from datasets import load_dataset
import generate_two_tower_data
import pandas as pd
import sp_tokenizer
import torch
import two_tower_model
import wandb
import torch.optim as optim
from tqdm import tqdm

In [8]:
# corpus = load_corpus.load_corpus()

In [9]:
# dataset = load_dataset("ms_marco", "v1.1")
# train_data = dataset["train"]
# train_df = pd.DataFrame(train_data)

In [10]:
# ds, passage_dict = generate_two_tower_data.generate_two_tower_data(train_df, 'ms_marco_model.model.model')

In [11]:
# torch.save(ds,'two_tower_training_data.pt')

In [ ]:
# ds = torch.load('two_tower_training_data.pt', map_location='cpu')




In [2]:
def custom_collate_fn(batch):
    # Sort the batch by target sequence length (in descending order)
    batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

    # Get the sequence lengths for both target and context
    target_seq_lengths = [len(item[0]) for item in batch]
    context_seq_lengths = [len(item[1]) for item in batch]

    # Pad target and context sequences separately
    padded_target_sequences = torch.nn.utils.rnn.pad_sequence([item[0] for item in batch], batch_first=True)
    padded_context_sequences = torch.nn.utils.rnn.pad_sequence([item[1] for item in batch], batch_first=True)

    # Create masks for the padded values for both target and context
    target_mask = (padded_target_sequences != 0).float()
    context_mask = (padded_context_sequences != 0).float()

    # Extract labels
    labels = [item[2] for item in batch]

    return torch.LongTensor(padded_target_sequences), torch.LongTensor(padded_context_sequences), torch.LongTensor(labels)


In [3]:
tokenizer = sp_tokenizer.sp_tokenizer()
def sp_decode(vectors,tokenizer):
    print(tokenizer.decode(vectors[0].tolist()), '\n',tokenizer.decode(vectors[1].tolist()),'\n', vectors[2].tolist())

In [4]:
# ds_part = ds[:20000]
# torch.save(ds_part, 'two_tower_training_section_of_data.pt')
# ds_part = torch.load('two_tower_training_section_of_data.pt')
# ds_part = torch.load('two_tower_training_section_of_data.pt')
ds = torch.load('two_tower_training_data.pt', map_location = 'cpu')
batch_size = 512
dl = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)


In [5]:
vocab_size = 16000  # Vocabulary size
embedding_dim = 100  # Dimension of word embeddings
hidden_dim = 128  # GRU hidden state size
output_dim = 64  # Desired output embedding dimension

In [6]:
model_GRU = two_tower_model.GRUEmbeddingModel(vocab_size, embedding_dim, hidden_dim, output_dim)
model_LSTM = two_tower_model.LSTMEmbeddingModel(vocab_size, embedding_dim, hidden_dim, output_dim)
cosine_similarity_loss = two_tower_model.CosineSimilarityLoss(margin = 0.2)
optimizer = optim.SGD(list(model_GRU.parameters()) + list(model_LSTM.parameters()), lr=0.01)


In [7]:
# import torch.nn.functional as F

# for query_0, doc_0, label_0 in dl:

#     query_emb = model_GRU(query_0)
#     doc_emb = model_LSTM(doc_0)
#     loss,predictions = cosine_similarity_loss(query_emb,doc_emb,label_0)
#     cos_sim = F.cosine_similarity(query_emb, doc_emb)
#     print(query_emb.size(), '\n', doc_emb.size(),'\n',loss,'\n',predictions, '\n',cos_sim)
#     break

In [8]:
# import torch.nn as nn
# embd_unit = nn.Embedding(vocab_size, embedding_dim)
# gru_unit = nn.GRU(embedding_dim,hidden_dim)
# fc_unit = nn.Linear(hidden_dim,output_dim)
# output, hidden = gru_unit(embd_unit(query_0))
# output = fc_unit(output)
# output = torch.mean(output, dim=1)

# print(output.size(), hidden.size())

In [7]:
wandb.login(key="9091f9245cf64052fa6d4eae03190a076fd87fe8")
wandb.init(project ='mlx_search_eninge', entity = 'basharkabalan',     config={
    "learning_rate": 0.01,
    "architecture": "two_tower",
    "dataset": "ms_marco_v1.1",
    "epochs": 3,
    })



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: basharkabalan. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\basha/.netrc


In [8]:
checkpoint = torch.load('model_checkpoint.pth')
model_GRU.load_state_dict(checkpoint['model_GRU_state_dict'])
model_LSTM.load_state_dict(checkpoint['model_LSTM_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
loss = checkpoint['loss']
correct_predictions = checkpoint['correct_predictions']
total_predictions = checkpoint['total_predictions']


In [10]:
num_epochs = 20


if last_epoch: starting_epoch = last_epoch

for epoch in range(last_epoch,num_epochs):
    correct_predictions = 0
    total_predictions = 0
    for query, doc, label in dl:
        optimizer.zero_grad()
        query = model_GRU(query)
        doc = model_LSTM(doc)
        # query = query.view(1,-1)
        # doc = doc.view(1,-1)
        loss,predictions = cosine_similarity_loss(query,doc,label)
        correct_predictions += torch.sum(predictions==label)
        total_predictions += batch_size
        acc = correct_predictions/total_predictions
        loss.backward()
        optimizer.step()


        wandb.log({"acc":acc,"loss": loss})
    print(f"Epoch {epoch+1}/{num_epochs}, Acc: {acc}, Loss: {loss.item()}")

Epoch 3/20, Acc: 0.9065716862678528, Loss: 0.2069016993045807
Epoch 4/20, Acc: 0.9157730340957642, Loss: 0.18329264223575592
Epoch 5/20, Acc: 0.9196393489837646, Loss: 0.21622639894485474
Epoch 6/20, Acc: 0.9206510782241821, Loss: 0.1901354193687439
Epoch 7/20, Acc: 0.9214475750923157, Loss: 0.19918417930603027
Epoch 8/20, Acc: 0.921684205532074, Loss: 0.19366173446178436
Epoch 9/20, Acc: 0.9219681620597839, Loss: 0.19390825927257538
Epoch 10/20, Acc: 0.9222632646560669, Loss: 0.19660376012325287
Epoch 11/20, Acc: 0.9225139617919922, Loss: 0.19360561668872833
Epoch 12/20, Acc: 0.9226958751678467, Loss: 0.17724639177322388


KeyboardInterrupt: 

In [11]:
# # Save the model parameters and optimizer state
# torch.save({
#     'epoch': epoch,
#     'model_GRU_state_dict': model_GRU.state_dict(),
#     'model_LSTM_state_dict': model_LSTM.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
#     'loss': loss,
#     'correct_predictions': correct_predictions,
#     'total_predictions': total_predictions
# }, 'model_checkpoint.pth')


In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CosineSimilarityLoss(nn.Module):
    def __init__(self, margin=0.0):
        super(CosineSimilarityLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, target):
        # Calculate cosine similarity
        cosine_similarity = F.cosine_similarity(output1, output2)
        print(cosine_similarity)

        # Calculate loss with margin
        loss = 1.0 - cosine_similarity + self.margin
        loss = torch.clamp(loss, min=0.0)  # Use ReLU to ensure positive loss

        # Multiply by target to penalize dissimilarity
        loss = target * (1-cosine_similarity) + (1-target) * (1+cosine_similarity)

        return loss.mean()

# Usage:
output1 = torch.tensor([1, 1], dtype=torch.float32)
output2 = torch.tensor([-1,1], dtype=torch.float32)
target = torch.tensor([1])  # 1.0 for similar, 0.0 for dissimilar

loss_fn = CosineSimilarityLoss(margin=0)
loss = loss_fn(output1.view(1,-1), output2.view(1,-1), target)

loss

tensor([0.])


tensor(1.)